In [1]:
import ast
import numpy as np
import torch
import matplotlib.pyplot as plt

from constants import DEVICE
from modules.data_utils import H5LazyDataset, DataUtils, MultiPipeline

In [ ]:
h5_lazy_dataset = H5LazyDataset(
    'data/train/train.h5', 'waveforms', 'labels',
    data_transforms=(
        lambda data: torch.tensor(data, dtype=torch.complex64, device=DEVICE),
        torch.fft.fft,
        torch.abs,
        lambda data: data.to(dtype=torch.float64),
        DataUtils.min_max_normalize,
        torch.fft.fftshift,
    ),
    label_transforms=(
        lambda _bytes: _bytes.decode('utf-8'),
        ast.literal_eval,
        lambda label: torch.tensor(label, dtype=torch.float64, device=DEVICE),
        MultiPipeline.split(2),
        MultiPipeline.split(2),
        MultiPipeline.apply(
            DataUtils.identity,
            MultiPipeline.apply(
                MultiPipeline.split(3),
                DataUtils.identity,
            ),
        ),
        MultiPipeline.flatten
    ),
)

In [19]:
_, label = h5_lazy_dataset[1]
print(label)

MultiPipeline(
	tensor([[2432., 2442.],
        [2402., 2412.]], device='cuda:0', dtype=torch.float64),
	tensor([[2432., 2442.],
        [2402., 2412.]], device='cuda:0', dtype=torch.float64),
	tensor([[2432., 2442.],
        [2402., 2412.]], device='cuda:0', dtype=torch.float64),
	tensor([[2432., 2442.],
        [2402., 2412.]], device='cuda:0', dtype=torch.float64),
	tensor([[2432., 2442.],
        [2402., 2412.]], device='cuda:0', dtype=torch.float64),
	tensor([[2432., 2442.],
        [2402., 2412.]], device='cuda:0', dtype=torch.float64)
)
